In [15]:
import numpy as np
import pandas as pd
import glob
import os
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

Columns of interest:
Month, DayOfMonth, DayOfWeek,Operating_Airline,Origin, OriginCity, OriginState,Dest,DestCityName,
DestState,DepTime,DepDelay,DepDel15,DepartureDelayGroups,TaxiOut,TaxiIn,ArrTime,ArrDelay,ArrDel15,
ArrivalDelayGroups,Cancelled,Diverted,ActualElapsedTime,AirTime,Distance,DistanceGroup,
CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay

In [ ]:
df = pd.read_csv('D:/Summer Practicum/1_Data/training_airlines.csv')
df.columns = df.columns.str.replace(' ', '')

In [27]:
model_oneway = smf.ols("DepDelay ~C(Operating_Airline)", data=df).fit()
model_oneway.summary()

sm.stats.anova_lm(model_oneway,type=2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Operating_Airline),9.0,4.140323e+07,4.600358e+06,1632.10993,0.0
Residual,3371655.0,9.503540e+09,2.818657e+03,NaN,NaN


In [26]:
model = smf.ols("DepDelay ~C(Operating_Airline) + C(OriginCityName)+ C(Operating_Airline):C(OriginCityName)", data=df).fit()
model.summary()

sm.stats.anova_lm(model,type=2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Operating_Airline),9.0,4.140323e+07,4.600358e+06,1641.185882,0.0
C(OriginCityName),239.0,2.403827e+07,1.005785e+05,35.881568,0.0
C(Operating_Airline):C(OriginCityName),2151.0,3.637355e+07,1.691006e+04,6.032695,0.0
Residual,3370696.0,9.448296e+09,2.803070e+03,NaN,NaN


In [40]:
pvals = model.pvalues
pvals.nlargest(15)

C(OriginCityName)[T.Harrisburg, PA]           0.996434
C(OriginCityName)[T.Jacksonville, FL]         0.992362
C(OriginCityName)[T.Portsmouth, NH]           0.992074
C(OriginCityName)[T.Martha's Vineyard, MA]    0.990589
C(OriginCityName)[T.Burlington, VT]           0.989325
C(OriginCityName)[T.Appleton, WI]             0.988467
C(OriginCityName)[T.Fort Myers, FL]           0.988314
C(OriginCityName)[T.Syracuse, NY]             0.984820
C(OriginCityName)[T.Santa Ana, CA]            0.982357
C(OriginCityName)[T.Seattle, WA]              0.975758
C(OriginCityName)[T.Charleston, SC]           0.973580
C(OriginCityName)[T.State College, PA]        0.969282
C(OriginCityName)[T.Madison, WI]              0.967286
C(OriginCityName)[T.Atlanta, GA]              0.966615
C(OriginCityName)[T.Houston, TX]              0.966506
dtype: float64

In [41]:
pvals.nsmallest(15)

C(Operating_Airline)[T.AS]                   0.000000e+00
C(Operating_Airline)[T.DL]                   0.000000e+00
C(Operating_Airline)[T.UA]                   0.000000e+00
C(Operating_Airline)[T.B6]                  1.195300e-212
C(Operating_Airline)[T.F9]                  2.024995e-165
C(Operating_Airline)[T.WN]                   4.081766e-83
C(Operating_Airline)[T.G4]                   5.788379e-41
C(Operating_Airline)[T.NK]                   1.970692e-13
C(Operating_Airline)[T.HA]                   1.196297e-12
C(OriginCityName)[T.San Luis Obispo, CA]     1.192218e-10
C(OriginCityName)[T.Gunnison, CO]            6.446050e-06
C(OriginCityName)[T.Pago Pago, TT]           1.374063e-05
C(OriginCityName)[T.Shreveport, LA]          1.369670e-03
C(OriginCityName)[T.Idaho Falls, ID]         2.747850e-03
C(OriginCityName)[T.Duluth, MN]              4.544930e-03
dtype: float64

In [31]:
model_nointeract = smf.ols("DepDelay ~C(Operating_Airline) + C(OriginCityName)", data=df).fit()
model_nointeract.summary()

sm.stats.anova_lm(model_nointeract,type=2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Operating_Airline),9.0,4.140323e+07,4.600358e+06,1636.132675,0.0
C(OriginCityName),239.0,2.403827e+07,1.005785e+05,35.771089,0.0
Residual,3371416.0,9.479501e+09,2.811727e+03,NaN,NaN


In [42]:
top_10_airports=['ATL','DFW','DEN','ORD','LAX','CLT','MCO','LAS','PHX','MIA']
df_top_10 = df[((df['Origin'].isin(top_10_airports)) | (df['Dest'].isin(top_10_airports)))]

In [44]:
model_top10 = smf.ols("ArrDelay ~C(Operating_Airline) + C(OriginCityName)+ C(Operating_Airline):C(OriginCityName)", data=df_top_10).fit()
model_top10.summary()

sm.stats.anova_lm(model_top10,type=2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Operating_Airline),9.0,3.884784e+07,4.316427e+06,1303.521261,0.0
C(OriginCityName),194.0,1.684903e+07,8.685067e+04,26.228103,0.0
C(Operating_Airline):C(OriginCityName),1746.0,2.687377e+07,1.539162e+04,4.648128,0.0
Residual,2009173.0,6.653093e+09,3.311359e+03,NaN,NaN


In [56]:
top_50_airports=df['Dest'].value_counts().nlargest(50).reset_index()
df_top_50 = df[(df['Dest'].isin(top_50_airports['Dest']))]

In [57]:
model_top50 = smf.ols("ArrDelay ~C(Operating_Airline) + C(OriginCityName)+ C(Operating_Airline):C(OriginCityName)", data=df_top_50).fit()
model_top50.summary()

sm.stats.anova_lm(model_top50,type=2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Operating_Airline),9.0,5.061188e+07,5.623543e+06,1835.808027,0.0
C(OriginCityName),213.0,2.324168e+07,1.091159e+05,35.620923,0.0
C(Operating_Airline):C(OriginCityName),1917.0,3.493685e+07,1.822475e+04,5.949479,0.0
Residual,2832195.0,8.675727e+09,3.063252e+03,NaN,NaN


In [79]:
pvals_top50 = model_top50.pvalues.reset_index(name="pvalues")
print(pvals_top50["pvalues"].nsmallest(10))

output = pvals_top50[pvals_top50['pvalues'] <= 0.05]
output

389     0.431012
1310    0.462295
130     0.462295
632     0.472971
1667    0.473733
116     0.474354
1991    0.477128
910     0.478125
1027    0.478125
505     0.478125
Name: pvalues, dtype: float64


,index,pvalues
